In [6]:
def page_Detail_104(detailurl):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import numpy as np
    aa=[]
#     link =[]
#     title=[]
#     corp=[]
#     area=[]
#     educational=[]
#     experience=[]
#     salary=[]
#     dt=[]
    from datetime import datetime
    res = requests.get(detailurl)
    soup = BeautifulSoup(res.text,"lxml")
    for news in soup.select('.b-block__left'):
        try:
            if True:
                link1 ='http:'+news.select_one('a.js-job-link').get('href')
                title1 = news.select_one('a.js-job-link').text.strip()
                corp1 = news.select_one('li a').text.strip()
                area1 = news.select_one('ul.job-list-intro li').text.strip()
                educational1=news.select_one('.job-list-intro').contents[9].text
                experience1=news.select_one('.job-list-intro').contents[5].text
                salary1=news.select_one('.b-tag--default').text
                dt1=news.select_one('span.b-tit__date').text.strip()
                aa.append({
                    'title':title1,
                    'dt':dt1,
                    'link':link1,
                    'area':area1,
                    'corp':corp1,
                    'salary':salary1,
                    'experience':experience1,
                    'educational':educational1,
                })
        except:
            pass
    return aa

In [7]:
def crawling_104(a,n=3):
    import pandas as pd
    import os
    import time
    newsary = []
    for page in range(n):
        url2 =('https://www.104.com.tw/jobs/search/?ro=0&keyword='+str(a)+'&area=6001001000%2C6001002000&order=12&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(page))
        for dic in (page_Detail_104(url2)):
            newsary.append(dic)
    df = pd.DataFrame(newsary)
    #對文字拆解
    df[['city','area1']] = df['area'].str.extract('(.+)市(.+)區')
    df['city']=df['city']+str('市')
    df['area1'] = df['area1']+str('區')
    df['experience']=df['experience'].str.extract('(\d+?)年以上')
    df['experience'].fillna(0,inplace=True)
    df['experience'] = df['experience'].map(lambda e: int(e))
    df[['sal_low','sal_high']]=df['salary'].map(lambda e: e.replace('月薪','').replace('元','').replace('待遇面議','0~0').replace(',','')).str.extract('(\d+)~(\d+)')
    df.drop(['area','salary'],axis=1,inplace = True)
    df['experience'].fillna('無經驗可',inplace=True)
    df['remark'] = '104人力'
    df2 = df[['title','corp','experience','educational','city','area1','dt','sal_low','sal_high','link','remark']]
    df2=df2.sort_values(by=['experience'],ascending=[False])
    return df2

In [8]:
def page_Detail_1111(detailurl):    
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    res = requests.get(detailurl)
    soup = BeautifulSoup(res.text)
    aa =[]
    bb =[]
    for news in soup.select('div.jbInfoin'):
        try:
            if True:
                title1 = news.select_one('a').get('title')
                link1 = 'https:'+news.select_one('a').get('href')
                corp1 = news.select_one('h4 a').text
                area1 = news.select_one('span.location').text
                educational1 = news.select_one('div.needs').contents[4]
                experience1 = news.select_one('div.needs').contents[2]
                salary1 = news.select_one('div.needs').contents[0].text
                aa.append({  
                    'title':title1,
                    'link':link1,
                    'area':area1,
                    'corp':corp1,
                    'salary':salary1,
                    'experience':experience1,
                    'educational':educational1,
                            })
        except:
            pass
    for dtime in soup.select('div.jbControl'):
        dt1 = dtime.select_one('div.date').text
        bb.append({'dt':dt1})
        
    return aa,bb

In [9]:
def crawling_1111(a,n=3):
    import pandas as pd
    import os
    import time
    newsary = []
    for page in range(n):
        url2 =('https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks='+str(a)+'&c0=100100,100200&page={}'.format(page))
        for dic in (page_Detail_1111(url2)):
            newsary.append(dic)
    df = pd.DataFrame(newsary[0])
    df1= pd.DataFrame(newsary[1])
    df = pd.concat([df,df1],axis = 1)
    df[['city','area1']] = df['area'].str.extract('(.+)市(.+)區')
    df['city']=df['city']+str('市')
    df['area1'] = df['area1']+str('區')
    df[['sal_low','sal_high']]=df['salary'].map(lambda e: e.replace('月薪','').replace('元','').replace('面議（經常性薪資4萬含以上）','0~0').replace(',','')).str.extract('(\d+)~(\d+)')
    df['experience']=df['experience'].str.extract('(\d+)')
    df['experience'].fillna(0,inplace=True)
    df['experience'] = df['experience'].map(lambda e: int(e))
    df['educational']=df['educational'].str.extract('(..)')
    df['educational'].replace('學歷','學歷不拘').replace('不拘','學歷不拘')
    df['dt']=df['dt'].str.slice(5,7)+'/'+df['dt'].str.slice(8,10)
    df['remark'] ='1111人力'
#     print(type(df['dt']))
    df3 = df[['title','corp','experience','educational','city','area1','dt','sal_low','sal_high','link','remark']]
    df3=df3.sort_values(by=['experience'],ascending=[False])
    return df3

In [18]:
import pandas as pd
import os
import time
SaveDirectory = os.getcwd()
con = str(input('請輸入職稱：'))
num = int(input('請輸入頁數(1-30)：'))
SaveAs = os.path.join(SaveDirectory,'work_crawl'+'_'+str(con)+ time.strftime('%Y_%m_%d_%H_%M') + '.xlsx')
if num <= 0:
    print('數字不可小於1')
elif num>30:
    c104=crawling_104(a=con,n=30)
    c1111=crawling_1111(a=con,n=30)
else:
    c104=crawling_104(a=con,n=num)
    c1111=crawling_1111(a=con,n=num)
    
df = pd.concat([c104,c1111],axis=0)
df.reset_index(inplace=True,drop=True) 
df[['mon','day']]=df['dt'].str.extract('(\d+)/(\d+)')
df['mon']=df['mon'].fillna(0)
df['day']=df['day'].fillna(0)
df['mon']=df['mon'].map(lambda e: int(e))
df['day']=df['day'].map(lambda e: int(e))
#df[['title','corp','experience','educational','city','area1','dt','sal_low','sal_high','link','remark','mon','day']]
df =df.sort_values(by=['mon','day','experience'],ascending=[True,True,False])
df2=df[['title','corp','experience','educational','city','area1','dt','sal_low','sal_high','link','remark']]
df2.to_excel(SaveAs,sheet_name='Work_Data',encoding='utf-8',startcol=1,startrow=1,index=False)


請輸入職稱：大數據資料分析師
請輸入頁數(1-30)：10
